![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import math

import time
from datetime import datetime

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
# сделаем то же для других столбцов
data['Cuisine_Style_isNAN']= pd.isna(data['Cuisine Style']).astype('uint8')
data['Price Range_isNAN']= pd.isna(data['Price Range']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN'].value_counts()

In [ ]:
# Далее заполняем пропуски средним значением по городу:

cities = list(data['City'].unique())
mean_NoR_citi = {}
for n in cities:
    if n in mean_NoR_citi:
        continue
    else:
        mean_NoR_citi[n] = data[data['City'] == n]['Number of Reviews'].mean()
        
right_views = []
for n,m in zip(data['City'], data['Number of Reviews']):
    if math.isnan(m):
        right_views.append(mean_NoR_citi[n])
    else:
        right_views.append(m)

data['Number of Reviews'] = right_views

Пропуски в столбце Cuisine Style заполним строкой 

In [ ]:
#пропуски в столбце Cuisine Style заполним строкой, т.к. будем позже считать кол-во кухонь
data['Cuisine Style'].fillna('unknown', inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
data.head(5)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

In [ ]:
values = {'$$$$': 3, '$' : 1, '$$ - $$$': 2, np.nan: np.nan}
data['Price Range'] = data['Price Range'].apply(lambda x: values[x])

In [ ]:
data['Price Range'].fillna(data['Price Range'].mean(), inplace=True)

In [ ]:
data['Price Range'].value_counts()

In [ ]:
# столбец City нам понадобится, поэтому преобразуем его позже
#data = pd.get_dummies(data, columns=[ 'City','Price Range'], dummy_na=True)

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
#посчитаем, сколько представлено кухонь, добавим признак
data['Cuisine Style'] = data['Cuisine Style'].apply(
    lambda x: str(x).replace('[', '').replace(']', '').replace("'", '').replace(' ', '').replace('"', ''))
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.split(','))
data['Number_of_kitchen'] = data['Cuisine Style'].apply(lambda x: len(x))

In [ ]:
# Добавим стобцы для кухонь и создадим для этого отдельный датасет
cuisines = []
for n in data['Cuisine Style']:
    if type(n) == float:
        continue
    else:
        for m in n:
            cuisines.append(m)
            
cuisine_set = set(cuisines)

df_cuisines = pd.DataFrame()
def find_item(cell):
    if item in cell:
        return 1
    return 0
for item in cuisine_set:
    df_cuisines[item] = data['Cuisine Style'].apply(find_item)

In [ ]:
# Добавим столбцы для городов, создадим отдельный датафрейм для него, а объединим позже
city_list = []
for n in data['City']:
    city_list.append(n)
cities = set(city_list)

df_cities = pd.DataFrame()
def find_item(cell):
    if item in cell:
        return 1
    return 0
for item in cities:
    df_cities[item] = data['City'].apply(find_item)

In [ ]:
data['Reviews']

In [ ]:
# Займемся датами отзывов. Значения даты представлены в формате str, переведем в формат float с помощью библиотеки datetime
# Переведем сначала их в формат YYYY/MM/DD, а затем в формат float. На этом шаге найдем среднее значение дат, 
# среднее значение времени между отзывами (mean_date_gap), максимальное значение между отзывами(max_date_gap):
reviews = data['Reviews']
dates = []
date_gap = []
for n in reviews:
    if type(n) != float:
        tmp_review = n.split('], [')
        tmp_date = tmp_review[1].replace("'", "").replace(']]', "").split(', ')
        if len(tmp_date) == 2:
            date_1 = tmp_date[0][6:10] + '/' + tmp_date[0][0:2] + '/' + tmp_date[0][3:5]
            date_1 = datetime.strptime(date_1, '%Y/%m/%d').timestamp()
            date_2 = tmp_date[1][6:10] + '/' + tmp_date[1][0:2] + '/' + tmp_date[1][3:5]
            date_2 = datetime.strptime(date_2, '%Y/%m/%d').timestamp()
            dates.append(date_1)
            date_gap.append(abs(date_1 - date_2))
            dates.append(date_2)
        elif len(tmp_date) == 1:
            if len(tmp_date[0]) > 0:
                date_1 = tmp_date[0][6:10] + '/' + tmp_date[0][0:2] + '/' + tmp_date[0][3:5]
                date_1 = datetime.strptime(date_1, '%Y/%m/%d').timestamp()
                dates.append(date_1)
            else:
                continue
    
mean_date = sum(dates) / len(dates)
max_date_gap = max(date_gap)
mean_date_gap = sum(date_gap) /len(date_gap)

# Повторим код, но уже заменяя нули средним значением:

reviews = data['Reviews']
dates = []
date_gap = []
for n in reviews:
    if type(n) != float:
        tmp_review = n.split('], [')
        tmp_date = tmp_review[1].replace("'", "").replace(']]', "").split(', ')
        if len(tmp_date) == 2:
            date_1 = tmp_date[0][6:10] + '/' + tmp_date[0][0:2] + '/' + tmp_date[0][3:5]
            date_1 = datetime.strptime(date_1, '%Y/%m/%d').timestamp()
            date_2 = tmp_date[1][6:10] + '/' + tmp_date[1][0:2] + '/' + tmp_date[1][3:5]
            date_2 = datetime.strptime(date_2, '%Y/%m/%d').timestamp()
            if date_1 > date_2:
                dates.append(date_1)
                date_gap.append(date_1 - date_2)
            else:
                dates.append(date_2)
                date_gap.append(date_2 - date_1)
        elif len(tmp_date) == 1:
            if len(tmp_date[0]) > 0:
                date_1 = tmp_date[0][6:10] + '/' + tmp_date[0][0:2] + '/' + tmp_date[0][3:5]
                date_1 = datetime.strptime(date_1, '%Y/%m/%d').timestamp()
                dates.append(date_1)
            else:
                dates.append(mean_date)
            date_gap.append(max_date_gap)
    else:
        dates.append(mean_date)
        date_gap.append(mean_date_gap)

In [ ]:
# если даты две, то используем только свежую дату, чтобы совпало кол-во строк в датафрейме. Добавляем столбцы:
data['Last review date'] = dates
data['Date_gap'] = date_gap

In [ ]:
#является ли ресторан сетевым
net_rest = dict(data['Restaurant_id'].value_counts())
for x,y in net_rest.items():
    if y==1:
        net_rest[x]=0
    else:
        net_rest[x]=1
data['net_rest']=data['Restaurant_id'].map(net_rest)

In [ ]:
data['net_rest'].value_counts()

In [ ]:
# Находится ли ресторан в столице?
capitals = [
        'London', 'Paris', 'Madrid', 'Berlin', 'Rome', 'Prague', 'Lisbon',
        'Vienna', 'Amsterdam', 'Brussels', 'Stockholm', 'Budapest', 'Warsaw',
        'Dublin', 'Copenhagen', 'Athens', 'Oslo', 'Helsinki', 'Bratislava',
        'Luxembourg', 'Ljubljana', 'Edinburgh']
data['Capital'] = data['City'].apply(lambda x: 1 if x in capitals else 0)

In [ ]:
# Добавим население городов
    population = {
        'Paris': 2190327,
        'Stockholm': 961609,
        'London': 8908081,
        'Berlin': 3644826,
        'Munich': 1456039,
        'Oporto': 237591,
        'Milan': 1378689,
        'Bratislava': 432864,
        'Vienna': 1821582,
        'Rome': 4355725,
        'Barcelona': 1620343,
        'Madrid': 3223334,
        'Dublin': 1173179,
        'Brussels': 179277,
        'Zurich': 428737,
        'Warsaw': 1758143,
        'Budapest': 1752286,
        'Copenhagen': 615993,
        'Amsterdam': 857713,
        'Lyon': 506615,
        'Hamburg': 1841179,
        'Lisbon': 505526,
        'Prague': 1301132,
        'Oslo': 673469,
        'Helsinki': 643272,
        'Edinburgh': 488100,
        'Geneva': 200548,
        'Ljubljana': 284355,
        'Athens': 664046,
        'Luxembourg': 115227,
        'Krakow': 769498
    }

    data['Population'] = data['City'].map(population)

In [ ]:
# Создадим новый столбец с количеством кухонь в разрезе городов и ресторанов
data['Cuisine_in_Restaurant_City'] = data['Cuisine Style'].apply(lambda x: len(str(x).split(', ')))

# Выделим в отдельную таблицу для удобства и посчитаем
# Количество ресторанов в каждом городе, количество кухонь в каждом городе
tmp_data = data[['City','Restaurant_id','Cuisine Style']].copy() 
tmp_data['Cuisine Style'] = tmp_data['Cuisine Style'].str.split(', ') 
tmp_data = tmp_data.explode('Cuisine Style')
tmp_data = tmp_data.copy()
tmp_data.sample(5)

# Группировка и расчет количества ресторанов в каждом городе, количество кухонь в каждом городе
tmp_data = tmp_data.groupby(['City'])[['Restaurant_id','Cuisine Style']].nunique()
tmp_data = tmp_data.reset_index()
tmp_data.columns = ['City','Restaurant_in_City','Cuisine_in_City']
tmp_data.sample(5)

data = data.merge(tmp_data, on = 'City')

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


In [ ]:
rest_number = data.groupby('City')['Restaurant_id'].count().reset_index()
rest_number.sort_values(by= 'Restaurant_id')
rest_number.rename(columns = {'Restaurant_id':'count_in_city'}, inplace = True)
rest_number

In [ ]:
data = data.merge(rest_number, on='City', how='left',sort=False)

In [ ]:
data['Ranking_norm']= data['Ranking']/data['count_in_city']
data['Ranking']= data['Ranking_norm']
data.drop(['Ranking_norm'],axis = 1, inplace=True)

In [ ]:
data.head()

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City','Price Range'], dummy_na=True)

### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['ID_TA',], axis = 1, inplace=True)
   
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Cuisine Style'].fillna("['Other']", inplace=True)
    
    values = {'$$$$': 3, '$' : 1, '$$ - $$$': 2, np.nan: np.nan}
    df_output['Price Range'] = df_output['Price Range'].apply(lambda x: values[x])
    df_output['Price Range'].fillna(df_output['Price Range'].mode()[0], inplace=True)
    
    #нормализуем признак Ranking,для этого сначала посчитаем кол-во ресторанов в городе
    rest_number= df_output.groupby('City')['Restaurant_id'].count().reset_index()
    rest_number.sort_values(by = 'Restaurant_id',ascending=False)
    rest_number.rename(columns = {'Restaurant_id':'count_in_city'}, inplace = True)

    df_output = df_output.merge(rest_number, on='City', how='left',sort=False)
    
    df_output['Ranking_norm']= df_output['Ranking']/df_output['count_in_city']# и найдем отношение Rank к кол-ву ресторанов
    df_output['Ranking']= df_output['Ranking_norm']
    df_output.drop(['Ranking_norm'],axis = 1, inplace=True)
    
    
    
    # ################### 3. Encoding ############################################################## 
    
    df_output = pd.get_dummies(df_output, columns=[ 'City','Price Range'], dummy_na=True)data['Cuisine Style'].fillna('unknown', inplace=True)
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(
    lambda x: str(x).replace('[', '').replace(']', '').replace("'", '').replace(' ', '').replace('"', ''))
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: x.split(','))
    df_output['Number_of_kitchen'] = df_output['Cuisine Style'].apply(lambda x: len(x))
    
    
    net_rest=dict(data['Restaurant_id'].value_counts())

    for x,y in net_rest.items():
        if y==1:
            net_rest[x]=0
    
        else:
            net_rest[x]=1
    data['net_rest']=data['Restaurant_id'].map(net_rest)        
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    #хочется попробовать стандартизировать признаки
    
    #def StandardScaler_column(df, d_col):
    #scaler = StandardScaler()
    #scaler.fit(df[[d_col]])
    #return scaler.transform(df[[d_col]])
    
    return df_output

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
